In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train_data=pd.read_csv('x_train.csv')
test_data=pd.read_csv('x_test.csv')

In [3]:
#train_data.head()

In [4]:
train_data.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)

In [5]:
#train_data.head()

In [6]:
test_data.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)

In [7]:
#test_data.head()

In [8]:
train_data=train_data.values
test_data=test_data.values

In [9]:
#train_data

In [10]:
from nltk.tokenize import word_tokenize

In [11]:
training_words=[]
testing_words=[]

In [12]:
for data in train_data:
    word=data[1]
    clean_word=word_tokenize(word)
    training_words.append((clean_word,data[0]))

for data in test_data:
    word=data[0]
    clean_word=word_tokenize(word)
    testing_words.append(clean_word)

In [13]:
#training_words

In [14]:
from nltk.corpus import stopwords
import string

In [15]:
stops=set(stopwords.words('english'))
punctuations=string.punctuation
stops.update(punctuations)
len(stops)

211

In [16]:
from nltk.corpus import wordnet #for pos start with part of speech tag
from nltk.stem import WordNetLemmatizer #lemmatize word with same meaning 
from nltk import pos_tag #part of speech tag

In [17]:
lemmatizer = WordNetLemmatizer()

In [18]:
a='better'
pos_tag([a]) #pos_tag[0][1] for getting part of speech

[('better', 'RBR')]

In [19]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [20]:
def clean_words(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w]) #for pos_tag in word rather than string
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [21]:
#training_words[0]

In [22]:
training_doc=[(clean_words(document),category) for document,category in training_words]

In [24]:
testing_doc=[clean_words(words) for words in testing_words]

In [ ]:
#training_doc[0][0]

In [25]:
#seperating words form tuple
all_words=[]
for word in training_doc:
    all_words+=word[0]

In [ ]:
#all_words

In [ ]:
#get top frequency words

In [26]:
import nltk
freq=nltk.FreqDist(all_words)
most_common_word=freq.most_common(10000)
features=[i[0] for i in most_common_word] # we want only word not their frequency

In [ ]:
#most_common_word ,features

In [27]:
def get_feature_dict(words): #get true or false values of features in all the documents
    curr_features={}
    word_set=set(words)
    for w in features: #above features
        curr_features[w]=w in word_set #add true value to that features which are in word_set otherwise false
    return curr_features

In [ ]:
#get_feature_dict(training_doc[0][0])

In [28]:
training_data=[(get_feature_dict(doc),category) for doc, category in training_doc]
testing_data=[(get_feature_dict(doc)) for doc in testing_doc]

In [35]:
from nltk.classify.scikitlearn import SklearnClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [36]:
svc=SVC()
rfc=RandomForestClassifier()

In [37]:
classifier_sklearn_svc=SklearnClassifier(svc)
classifier_sklearn_rfc=SklearnClassifier(rfc)

In [38]:
classifier_sklearn_svc.train(training_data)

<SklearnClassifier(SVC())>

In [41]:
predictions=[]
for doc in testing_data:
    predictions.append(classifier_sklearn_svc.classify(doc))

In [43]:
np.savetxt('pred.csv',predictions,fmt='%s')

In [49]:
#using random forest

In [51]:
classifier_sklearn_rfc.train(training_data)

<SklearnClassifier(RandomForestClassifier())>

In [52]:
predictions=[]
for doc in testing_data:
    predictions.append(classifier_sklearn_rfc.classify(doc))

In [53]:
np.savetxt('pred_rfc.csv',predictions,fmt='%s')